# Preprocessing
___

In [68]:
import pandas as pd
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

___
# Data Imports
We will import the listings, reviews and calendar data

In [69]:
df1 = pd.read_csv('../data/listings.csv.gz', compression = 'gzip')
df1.to_csv('../data/listings.csv')

In [70]:
df2 = pd.read_csv('../data/reviews.csv.gz', compression = 'gzip')
df2.to_csv('../data/reviews.csv')

In [71]:
df3 = pd.read_csv('../data/calendar.csv.gz', compression = 'gzip')
df3.to_csv('../data/calendar.csv')

___
# Preprocessing

In [72]:
# Check the shape first to see how much data we can reasonably drop 
df1.shape

(13327, 75)

In [73]:
# Here is more null value and dtype information
# This will aid in the choosing of predictor variables for a linear regression

null_info = pd.concat([df1.isnull().sum(), df1.dtypes], axis=1)
null_info.columns = ['Null Values', 'Data Type']
null_info['Column Name'] = null_info.index
null_info.reset_index(drop=True, inplace=True)

print(null_info)

    Null Values Data Type                                   Column Name
0             0     int64                                            id
1             0    object                                   listing_url
2             0     int64                                     scrape_id
3             0    object                                  last_scraped
4             0    object                                        source
..          ...       ...                                           ...
70            0     int64                calculated_host_listings_count
71            0     int64   calculated_host_listings_count_entire_homes
72            0     int64  calculated_host_listings_count_private_rooms
73            0     int64   calculated_host_listings_count_shared_rooms
74         3000   float64                             reviews_per_month

[75 rows x 3 columns]


In [74]:
# Many entries which should be numeric are object type 
# We see that it is because of inclusion of dollar signs and t/f instad of 1/0 booleans etc
# We will clean this up later
df1.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3176,https://www.airbnb.com/rooms/3176,20231218233220,2023-12-19,previous scrape,Rental unit in Berlin · ★4.63 · 1 bedroom · 2 ...,NaN,The neighbourhood is famous for its variety of...,https://a0.muscache.com/pictures/243355/84afcf...,3718,...,4.69,4.92,4.62,First name and Last name: Nicolas Krotz <br/> ...,f,1,1,0,0,0.84
1,9991,https://www.airbnb.com/rooms/9991,20231218233220,2023-12-19,city scrape,Rental unit in Berlin · ★5.0 · 4 bedrooms · 7 ...,NaN,Prenzlauer Berg is an amazing neighbourhood wh...,https://a0.muscache.com/pictures/42799131/59c8...,33852,...,5.00,4.86,4.86,03/Z/RA/003410-18,f,1,1,0,0,0.07
2,183988,https://www.airbnb.com/rooms/183988,20231218233220,2023-12-19,city scrape,Rental unit in Berlin · ★4.69 · 1 bedroom · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/1041e6fd-c369...,882801,...,4.79,4.72,4.62,04/Z/ZA/004232-16,f,1,1,0,0,3.92


In [75]:
# Should we use host_response_time?
# We will use ordinal encoder in the regression notebook for this 
df1['host_response_time'].value_counts()

host_response_time
within an hour        5188
within a few hours    1413
within a day          1308
a few days or more     366
Name: count, dtype: int64

___
# Making Dataframe For Regression Modeling

Now that we have checked many things about the data, we can select which features to use in a linear regression model. This will consist of choosing the columns from df1 we want to use, cleaning them by dropping special characters such as dollar signs so as to render them numeric, removing commas, making t/f columns 0 or 1 etc.

Using a priori knowledge of housing price, we can select the following predictors for price. Note that we also cross checked these choices with the null values table from above because, if there are too many, it will result in data loss.
* price (the response variable)
* host_response_time
* host_response_rate
* host_acceptance_rate
* host_is_superhost
* accommodates
* beds
* review_scores_rating
* review_scores_accuracy
* review_scores_cleanliness
* review_scores_checkin
* review_scores_communication
* review_scores_location
* review_scores_value

In [76]:
df = df1[[
    'price',
    'host_response_time',
    'host_response_rate',
    'host_acceptance_rate',
    'host_is_superhost',
    'accommodates',
    'beds',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]]

In [77]:
df.head()

,price,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,accommodates,beds,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,$83.00,within a day,100%,17%,f,4,2.0,4.63,4.67,4.52,4.64,4.69,4.92,4.62
1,$180.00,within a day,100%,NaN,f,7,7.0,5.00,5.00,5.00,5.00,5.00,4.86,4.86
2,$116.00,within a few hours,100%,99%,f,4,2.0,4.69,4.73,4.73,4.78,4.79,4.72,4.62
3,$70.00,within a few hours,100%,47%,f,1,1.0,4.68,5.00,4.85,4.70,4.85,4.60,4.45
4,$100.00,within a day,100%,100%,f,2,2.0,4.40,4.07,4.47,4.80,4.73,4.87,4.00


In [78]:
# 13,327 rows
df.shape

(13327, 14)

In [79]:
df.dropna(inplace = True)

In [80]:
# We lost over half of our data by dropping null values but we still have a lot of data for just 14 columns
df.shape

(6202, 14)

In [81]:
# Now we clean up these columns

df.loc[:, 'price'] = df['price'].str.replace('[$,]', '', regex=True)
df['price'] = df['price'].astype('float64')
df.loc[:, 'host_response_rate'] = df['host_response_rate'].str.replace('%', '')
df.loc[:, 'host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('%', '')
df['host_response_rate'] = df['host_response_rate'].astype('float64')
df['host_acceptance_rate'] = df['host_acceptance_rate'].astype('float64')
df.loc[:, 'host_is_superhost'] = df['host_is_superhost'].map({'t':1, 'f':0})
df['host_is_superhost'] = df['host_is_superhost'].astype('float64')

In [82]:
# Looks good now 
df.dtypes

price                          float64
host_response_time              object
host_response_rate             float64
host_acceptance_rate           float64
host_is_superhost              float64
accommodates                     int64
beds                           float64
review_scores_rating           float64
review_scores_accuracy         float64
review_scores_cleanliness      float64
review_scores_checkin          float64
review_scores_communication    float64
review_scores_location         float64
review_scores_value            float64
dtype: object

In [83]:
df.head()

,price,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,accommodates,beds,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,83.0,within a day,100.0,17.0,0.0,4,2.0,4.63,4.67,4.52,4.64,4.69,4.92,4.62
2,116.0,within a few hours,100.0,99.0,0.0,4,2.0,4.69,4.73,4.73,4.78,4.79,4.72,4.62
3,70.0,within a few hours,100.0,47.0,0.0,1,1.0,4.68,5.00,4.85,4.70,4.85,4.60,4.45
4,100.0,within a day,100.0,100.0,0.0,2,2.0,4.40,4.07,4.47,4.80,4.73,4.87,4.00
8,25.0,within an hour,100.0,96.0,1.0,2,1.0,4.77,4.82,4.71,4.89,4.92,4.87,4.65


In [84]:
# Save this as a csv to be used for the regression models in 03_regression
df.to_csv('../data/03_data.csv')

___
# Making Dataframe For Forecasting

We need to make a data frame with date time as the index and a price column. We will have to aggregate all of the sales by date so that we can see total sales by date. This can be achieved with a simply groupby. We will use the price in the calendar dataset and filter by availability being false. The assumption here is that when the availability is false, it is being occupied and revenue is being collected.

In [85]:
# All we need for forecast modeling is date and price
fc = df3[['date', 'price']]
fc['date'] = pd.to_datetime(fc['date'])

In [86]:
# Only show the properties which are occupied
fc = df3[df3['available'] == 'f']

In [87]:
# We still have to aggregate sales across all properties by date
fc['date'].value_counts()

date
2023-12-19    10738
2024-09-28    10429
2024-09-27    10408
2023-12-31    10355
2023-12-30    10242
              ...  
2024-03-14     7207
2024-03-13     7177
2024-03-12     7165
2024-03-17     7125
2023-12-18       11
Name: count, Length: 366, dtype: int64

In [88]:
# Remove the dollar sign and commas to make price numeric
fc.loc[:, 'price'] = fc['price'].str.replace('[$,]', '', regex=True)

# Convert the modified values in the 'price' column to numeric, coercing invalid parsing to NaN
fc['price'] = fc['price'].astype('float64')

In [89]:
# Group by 'date' and sum the 'price' column
sales = fc.groupby('date')['price'].sum().reset_index()
sales

,date,price
0,2023-12-18,679.0
1,2023-12-19,1238727.0
2,2023-12-20,1064670.0
3,2023-12-21,1034584.0
4,2023-12-22,998590.0
...,...,...
361,2024-12-13,1159658.0
362,2024-12-14,1159189.0
363,2024-12-15,1159084.0
364,2024-12-16,1164876.0


In [90]:
# Now the price column is a float and can be graphed
sales.dtypes

date      object
price    float64
dtype: object

In [91]:
# Set the date as the index
sales.set_index('date', inplace=True)

In [92]:
# We will remove the first row because we only have a few sales on that day
# This is most likely due to data collection starting midday
sales = sales[1:]

In [93]:
# Looks good!
sales.head()

,price
date,
2023-12-19,1238727.0
2023-12-20,1064670.0
2023-12-21,1034584.0
2023-12-22,998590.0
2023-12-23,1055642.0


In [94]:
# Save to csv to be used for 04_forecasting 
sales.to_csv('../data/04_sales.csv')

___
# Making Dataframe For Sentiment Analysis

We have a concise DataFrame named df2 containing listing information. Our goal is to prepare individual reviews for analysis by implementing a function that cleans them up, ensuring proper punctuation and spacing. This preprocessing step is crucial for subsequent tasks.

The function we'll create will first address punctuation and spacing issues within each review. Next, it will tokenize the review, breaking it down into individual words or phrases. Finally, the function will apply stemming to these tokens, identifying the root forms of words.

In [95]:
# Function to preprocess text
def preprocess_text(text):
    # Punctuation 
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [96]:
df2.dropna(inplace = True)

In [97]:
df2.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3176,4283,2009-06-20,21475,Milan,"excellent stay, i would highly recommend it. a..."
1,3176,134722,2010-11-07,263467,George,Britta's apartment in Berlin is in a great are...
2,3176,144064,2010-11-24,76726,Patricia,"Fantastic, large place in good location. Only ..."
3,3176,156702,2010-12-21,291657,Benedetta,L'appartamento di Britta è molto largo carino ...
4,3176,165048,2011-01-04,279789,Aude,We went in Berlin for the new year eve. The ap...


In [98]:
# Apply the function 
df2['preprocessed_review'] = df2['comments'].apply(preprocess_text)

In [99]:
# Make a new dataframe with just listing ID and preprocessed review
df_rev = df2[['listing_id', 'preprocessed_review']]

In [100]:
df_rev.head()

,listing_id,preprocessed_review
0,3176,excel stay would highli recommend nice flat ni...
1,3176,britta apart berlin great area numer fantast r...
2,3176,fantast larg place good locat short tram ride ...
3,3176,lappartamento di britta è molto largo carino c...
4,3176,went berlin new year eve appart britta realli ...


In [101]:
# Save this to a dataframe to be used on 05_sentiment_analysis
df_rev.to_csv('../data/05_data.csv')